Problem1-SSOP

Problem is divided in to two subproblems (i) from start point to the signal intersection and (ii) from traffic signal intersection to end point

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Define constants
d0 = 0  # Initial position
v0 = 15  # Initial velocity (m/s)
vf=0 # final velocity(m/s)
df = 60  # Desired endpoint (m)

v_max = 22  # Maximum velocity (m/s)
v_min = 0   # Minimum velocity (m/s)
a_max = 8  # Maximum acceleration (m/s^2)
a_min = -5  # Minimum acceleration (m/s^2)
T_g = 25  # Time for the signal to change from green to yellow (s)
T_y = 5  # Time for the signal to change from yellow to red (s)
T_r = 26  # Time for the signal to change from red to green (s)
phi=28
dx = 2  # distance resolution (s)
m=1360.777  # mass of the vehicle
nd=0.9         #driveline efficiency 85%-95%
raw= 1.2256  #density of air
H=0.5
Cd=0.37
Ch=1-0.085*H
Cr=1.75
Af=1.98
c1= 0.0328
c2=4.575


# Cost function
alpha0=6.289*(10**(-4))
alpha1=2.676*10**(-5)
alpha2=10**(-6)

# Define state and action grids
t_grid = np.arange(0,T_g+T_y+T_r,0.1) # all possible value t_res can take, it will be used to capture the information of traffic signal dynamics
V_grid = np.arange(v_min, v_max+1,1)
A_grid = np.arange(a_min, a_max+1,0.1)

k=int(df/dx)

dest=(vf,0)  # End point
src=(v0,phi) # Start point


src_c=(np.argmin(np.abs(V_grid - src[0])),np.argmin(np.abs(t_grid - src[1])))     # starting coordinate in state space grid
dest_c=(np.argmin(np.abs(V_grid - dest[0])),np.argmin(np.abs(t_grid - dest[1])))  # end coordinate in state space grid

total_cost={(dest_c,k):0} # initialize value fuction
optimal_actions = {(dest_c,k):0}   # initialize optimal actions


# Value iteration
while k>0:
    k=k-1
    if k>50/dx:  # Subproblem 2, from traffic intersection to the end point
       for vi, v in enumerate(V_grid):
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v


                  next_vi=np.argmin(np.abs(V_grid - next_v))
                  next_ti=0;


                  if ((next_vi,next_ti),k+1) not in total_cost:
                      total_cost[((next_vi,next_ti),k+1)]=float('inf')


                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000 #

                  Pt=(Rt+1.04*m*a)*v/(3600*nd) # power

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0


                  immediate_cost = (Fct)*dt

                  expected_cost = immediate_cost + total_cost[((next_vi,next_ti),k+1)]
                  if expected_cost < min_cost:
                      min_cost = expected_cost
                      best_action = [((next_vi,next_ti),dt,a)]


              total_cost[((vi,0),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,0),k)] = best_action

    if k==50/dx: # at the intersection
       for vi, v in enumerate(V_grid):
        for ti, t in enumerate(t_grid):
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v


                  next_vi=np.argmin(np.abs(V_grid - next_v))
                  next_ti=0

                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000

                  Pt=(Rt+1.04*m*a)*v/(3600*nd)

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0




                  if ((next_vi,next_ti),k+1) not in total_cost:
                      total_cost[((next_vi,next_ti),k+1)]=float('inf')

                  if v!=0 and t>30:
                    immediate_cost=float('inf')
                  elif v==0 and t>30:
                    immediate_cost = (Fct)*dt+alpha0*(T_g+T_y+T_r-t)
                    dt=dt+T_g+T_y+T_r-t
                  else:
                    immediate_cost = (Fct)*dt

                  expected_cost = immediate_cost + total_cost[((next_vi,next_ti),k+1)]
                  if expected_cost < min_cost:
                      min_cost = expected_cost
                      best_action = [((next_vi,0),dt,a)]


              total_cost[((vi,ti),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,ti),k)] = best_action

    if k<50/dx:     # from start point to the end point
       for vi, v in enumerate(V_grid):
        for ti, t in enumerate(t_grid): # t is the t_res at any point of time
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v

                  next_t=(t+dt)%(T_g+T_y+T_r)

                  next_vi=np.argmin(np.abs(V_grid - next_v))
                  next_ti=np.argmin(np.abs(t_grid - next_t))


                  if ((next_vi,next_ti),k+1) not in total_cost:
                      total_cost[((next_vi,next_ti),k+1)]=float('inf')


                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000

                  Pt=(Rt+1.04*m*a)*v/(3600*nd)

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0


                  immediate_cost = (Fct)*dt

                  expected_cost = immediate_cost + total_cost[((next_vi,next_ti),k+1)]
                  if expected_cost < min_cost:
                      min_cost = expected_cost
                      best_action = [((next_vi,next_ti),dt,a)]


              total_cost[((vi,ti),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,ti),k)] = best_action




Forward motion for SSOP- As SSOP and SSSP take time to run, I have written this code which extract optimal acceleration from the lookup table.
In order to run this code, load the SSOP parameter data file (it contains optimal_actions lookup table) in SPYDER and run below code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

for phi in range(0,56,2): # phi is the all possible
    optimal_path=[]
    phi=phi+0.1
    vi=src_c[0]
    ti=np.argmin(np.abs(t_grid - phi))
    Vi=v0
    Ti=phi
    total_time=phi

    for i in range(0,int(df/dx)):
        if i>50/dx:
          ti=0
        ai=optimal_actions[((vi,ti),i)][0][2]
        total_time=total_time+optimal_actions[((vi,ti),i)][0][1]
        ti=optimal_actions[((vi,ti),i)][0][0][1]
        Vi=math.sqrt(Vi*Vi + 2*ai*dx)
        Ti=(total_time+phi)%(T_g+T_y+T_r)
        vi=np.argmin(np.abs(V_grid - Vi))
        Vi=V_grid[vi]
        optimal_path.append((Vi,dx*i,ai,total_time))


        data = np.array(optimal_path)
        x = data[:, 0]
        y = data[:, 1]
        z = data[:, 2]
        t = data[:, 3]

        plt.plot(t,y,"y-")



plt.plot(np.arange(0.1,25.1, 0.1),50*np.ones(250),"g-")
plt.plot(np.arange(25.1,30.1, 0.1),50*np.ones(50),"y-")
plt.plot(np.arange(30.1,T_g+T_y+T_r+0.1, 0.1),50*np.ones(260),"r-")
plt.xlabel("time")
plt.ylabel("Distance")
plt.title("SSOP")
plt.xticks(np.arange(0, 60, step=4))
plt.savefig('SSOP.eps', format='eps')




Problem 2 SSSP-Markov Decision process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Define constants
d0 = 0  # Initial position
v0 = 15  # Initial velocity (m/s)
vf=0 # final velocity(m/s)
df = 60  # Desired endpoint (m)

v_max = 22  # Maximum velocity (m/s)
v_min = 0   # Minimum velocity (m/s)
a_max = 8  # Maximum acceleration (m/s^2)
a_min = -5  # Minimum acceleration (m/s^2)
T_g = 25  # Time for the signal to change from green to yellow (s)
T_y = 5  # Time for the signal to change from yellow to red (s)
T_r = 26  # Time for the signal to change from red to green (s)
phi=28
dx = 2  # distance resolution (s)
m=1360.777  # mass of the vehicle
nd=0.9         #driveline efficiency 85%-95%
raw= 1.2256  #density of air
H=0.5
Cd=0.37
Ch=1-0.085*H
Cr=1.75
Af=1.98
c1= 0.0328
c2=4.575


# Cost function
alpha0=6.289*(10**(-4))
alpha1=2.676*10**(-5)
alpha2=10**(-6)

# Define state and action grids

tg_grid = np.arange(0,T_g,1) # all posible values of t_res for green signal
ty_grid = np.arange(0,T_y,1) # all posible values of t_res for yellow signal
tr_grid = np.arange(0,T_r,1) # all posible values of t_res for red signal
D_grid=np.arange(0,2,1) # Binary decision variable to decide if the change in signal is observed or not (0,1)- 1 - fully known dynamics
L_grid=np.arange(0,3,1) # 0-Green, 1-Yellow, 2-Red light signal
V_grid = np.arange(v_min, v_max+1,1)
A_grid = np.arange(a_min, a_max+1,0.1)

k=int(df/dx)

dest=(vf,1,0,0)  # By the end point the system dynamics will be fully known hence D=1
src=(v0,0,0,0) # Start point (velocity,det,traffic signal,time duration), at the start we don't know the system dynamics hence D=0

if src[2]==0:
  t_grid=tg_grid

elif src[2]==1:
  t_grid=ty_grid

else:
  t_grid=tr_grid


src_c=(np.argmin(np.abs(V_grid - src[0])),np.argmin(np.abs(D_grid - src[1])),np.argmin(np.abs(L_grid - src[2])),np.argmin(np.abs(t_grid - src[3])))     # starting coordinate in state space grid
dest_c=(np.argmin(np.abs(V_grid - dest[0])),1,0,0)  # end coordinate in state space grid

total_cost={(dest_c,k):0} # initialize value fuction
optimal_actions = {(dest_c,k):0}   # initialize optimal actions



# Value iteration
while k>0:
    k=k-1
    if k>50/dx: # subproblem (ii)
       for vi, v in enumerate(V_grid):
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v


                  next_vi=np.argmin(np.abs(V_grid - next_v))
                  next_ti=0
                  next_Di=1
                  next_Li=0

                  if ((next_vi,next_Di,next_Li,next_ti),k+1) not in total_cost:
                      total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]=float('inf')


                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000

                  Pt=(Rt+1.04*m*a)*v/(3600*nd)

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0


                  immediate_cost = (Fct)*dt

                  expected_cost = immediate_cost + total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]
                  if expected_cost < min_cost:
                      min_cost = expected_cost
                      best_action = [((next_vi,next_Di,next_Li,next_ti),dt,a)]


              total_cost[((vi,1,0,0),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,1,0,0),k)] = best_action

    if k==50/dx:  # at the trafic signal intersection - interface between problem (i) and (ii)
       for vi, v in enumerate(V_grid):
        for Di, D in enumerate(D_grid):
          for Li, L in enumerate(L_grid):
            if L==0:
              t_grid=tg_grid

            elif L==1:
              t_grid=ty_grid

            else:
              t_grid=tr_grid


            for ti, t in enumerate(t_grid):
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v


                  next_vi=np.argmin(np.abs(V_grid - next_v))
                  next_ti=0   # after crossing traffic signal we don't need any information related to traffic signal dyanmics hence this is assigned 0
                  next_Di=1
                  next_Li=0 # after crossing traffic signal we don't need any information related to traffic signal dyanmics hence this is assigned 0



                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000

                  Pt=(Rt+1.04*m*a)*v/(3600*nd)

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0

                  if ((next_vi,next_Di,next_Li,next_ti),k+1) not in total_cost:
                       total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]=float('inf')

                  if v!=0 and L==2:
                     immediate_cost=float('inf')
                  elif v==0 and L==2 and D==1:
                    immediate_cost = (Fct)*dt+alpha0*(T_r-t)
                    dt=dt+T_r-t
                  elif v==0 and L==2 and D==0:
                    immediate_cost = (Fct)*dt+alpha0*(T_r-t)/2
                    dt=dt+(T_r-t)
                  else:
                    immediate_cost = (Fct)*dt

                  expected_cost = immediate_cost + total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]
                  if expected_cost < min_cost:
                        min_cost = expected_cost
                        best_action = [((next_vi,next_Di,next_Li,next_ti),dt,a)]




              total_cost[((vi,Di,Li,ti),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,Di,Li,ti),k)] = best_action

    if k<50/dx:   # subproblem (i)
       for vi, v in enumerate(V_grid):
        for Di, D in enumerate(D_grid):
          for Li, L in enumerate(L_grid):
            if L==0:
              t_grid=tg_grid
              total_t=T_g

            elif L==1:
              t_grid=ty_grid
              total_t=T_y

            else:
              t_grid=tr_grid
              total_t=T_r


            for ti, t in enumerate(t_grid):
              min_cost = float('inf')
              best_action = None
              for ai, a in enumerate(A_grid):
                  if v*v + 2*a*dx>=0:
                    next_v = math.sqrt(v*v + 2*a*dx)

                  else:
                    next_v=-1
                  if next_v >22 or next_v<0:
                      continue

                  if a>0:
                    dt=(next_v-v)/a

                  elif a<0:
                    dt=(next_v-v)/a

                  else:
                    dt=dx/v


                  next_vi=np.argmin(np.abs(V_grid - next_v))

                  Rt=(raw/25.92)*Cd*Ch*Af*v*v+9.8066*m*Cr*(c1*v+c2)/1000

                  Pt=(Rt+1.04*m*a)*v/(3600*nd)

                  if(Pt>0):
                    Fct=alpha0+alpha1*Pt+alpha2*Pt**2
                  else :
                    Fct=alpha0

                  immediate_cost = (Fct)*dt

                  if D==1:

                    if t+dt>total_t and L==2:
                      next_Li=0
                    elif t+dt>total_t:
                      next_Li=L+1
                    else:
                      next_Li=Li

                    next_t=(t+dt)%total_t
                    next_ti=np.argmin(np.abs(t_grid - next_t))



                    if ((next_vi,1,next_Li,next_ti),k+1) not in total_cost:
                        total_cost[((next_vi,1,next_Li,next_ti),k+1)]=float('inf')




                    expected_cost = immediate_cost + total_cost[((next_vi,1,next_Li,next_ti),k+1)]
                    if expected_cost < min_cost:
                          min_cost = expected_cost
                          best_action = [((next_vi,next_Di,next_Li,next_ti),dt,a)]



                  else:

                    P1=(total_t-t-dt)/total_t # probability signal remain same
                    if (total_t-t-dt)<=0:
                      P1=0
                    P2=1-P1  # probability signal change


                    next_t=(t+dt)%total_t
                    next_ti=np.argmin(np.abs(t_grid - next_t))
                    next_Di=0

                    next_Li=Li




                    cost1=total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]

                    next_Di=1

                    if L==2:
                      next_Li=0
                    else:
                      next_Li=L+1


                    if ((next_vi,next_Di,next_Li,next_ti),k+1) not in total_cost:
                        total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]=float('inf')


                    cost2=total_cost[((next_vi,next_Di,next_Li,next_ti),k+1)]




                    expected_cost = immediate_cost + P1*cost1+P2*cost2
                    if expected_cost < min_cost:
                          min_cost = expected_cost
                          best_action = [((next_vi,next_Di,next_Li,next_ti),dt,a)]




              total_cost[((vi,Di,Li,ti),k)]=min_cost
              if min_cost<float('inf'):
                optimal_actions[((vi,Di,Li,ti),k)] = best_action



Forward Motion-SSSP As SSOP and SSSP take time to run, I have written this code which extract optimal acceleration from the lookup table.
In order to run this code, load the SSSP parameter data file (it contains optimal_actions lookup table) in SPYDER and run below code.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

for phi in range(0,56,2):
    phi=phi+0.1
    optimal_path=[]
    vi=src_c[0]
    Vi=v0
    Ti=phi
    total_time=phi
    t_obs=0
    if phi%(T_g+T_y+T_r)<T_g:
      l=0
    elif  T_g<=phi%(T_g+T_y+T_r)<T_g+T_y:
      l=1
    else:
      l=2

    D=0
    ti=0

    for i in range(0,int(df/dx)):
        if i>50/dx:
          ti=0
          D=1
          l=0
        ai=optimal_actions[((vi,D,l,ti),i)][0][2]
        dt=optimal_actions[((vi,D,l,ti),i)][0][1]

        v=Vi
        Vi=math.sqrt(Vi*Vi + 2*ai*dx)
        vi=np.argmin(np.abs(V_grid - Vi))
        next_v=V_grid[vi]

        if i==50/dx:
            a=ai

            if a>0:
                dt=(next_v-v)/a

            elif a<0:
                dt=(next_v-v)/a

            else:
                dt=dx/v

            if l==2 and D==0:
                dt=dt+T_r-t_obs-phi%(T_g+T_y)

            elif l==2 and D==1:
                dt=dt+T_r-t_obs



        total_time=total_time+dt
        t_obs=t_obs+dt

        if (total_time)%(T_g+T_y+T_r)<T_g:
          l1=0
        elif  T_g<=(total_time)%(T_g+T_y+T_r)<T_g+T_y:
          l1=1
        else:
          l1=2

        if l1!=l:
          l=l1
          D=1
          t_obs=0


        if l==0:
           t_grid=tg_grid
           total_t=T_g

        elif l==1:
            t_grid=ty_grid
            total_t=T_y

        else:
             t_grid=tr_grid
             total_t=T_r



        Ti=(total_time)%(T_g+T_y+T_r)

        ti=np.argmin(np.abs(t_grid - t_obs))
        Vi=V_grid[vi]
        optimal_path.append((Vi,dx*i,ai,total_time))
    data = np.array(optimal_path)
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]
    t = data[:, 3]



    plt.plot(t,y,'-y')


plt.plot(np.arange(0.1,25.1, 0.1),50*np.ones(250),"g-")
plt.plot(np.arange(25.1,30.1, 0.1),50*np.ones(50),"y-")
plt.plot(np.arange(30.1,T_g+T_y+T_r+0.1, 0.1),50*np.ones(260),"r-")
plt.xlabel("time")
plt.ylabel("Distance")
plt.title("SSSP")
plt.xticks(np.arange(0, 60, step=4))
plt.savefig('SSSP.eps', format='eps')




